In [1]:
import cv2

In [2]:
# خواندن تصویر
image = cv2.imread("TestImage.jpg")  # جایگزین با مسیر تصویر خودت

# بررسی اینکه تصویر لود شده یا نه
if image is None:
    print("Error: Image not found.")
    exit()

In [ ]:
# تبدیل به خاکستری
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# بهبود کنتراست با Equalization
equalized = cv2.equalizeHist(gray)

# نمایش تصاویر
cv2.imshow("Original Gray", gray)
cv2.imshow("Equalized Image", equalized)

# صبر تا فشار دادن کلید
cv2.waitKey(0)
cv2.destroyAllWindows()